# Solvers ⚙️

In this exercise, you will investigate the effects of different `solvers` on `LogisticRegression` models.

👇 Run the code below to import the dataset

In [1]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_dataset.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


- The dataset consists of different wines 🍷
- The features describe different properties of the wines 
- The target 🎯 is a quality rating given by an expert

## 1. Target engineering

In this section, you are going to transform the ratings into a binary target.

👇 How many observations are there for each rating?

In [5]:
# YOUR CODE HERE
df['quality rating'].unique()

array([ 6,  7,  4,  8,  3,  1,  2, 10,  5,  9])

In [11]:
df['quality rating'].value_counts()

10    10143
5     10124
1     10090
2     10030
8      9977
6      9961
9      9955
7      9954
4      9928
3      9838
Name: quality rating, dtype: int64

❓ Create `y` by transforming the target into a binary classification task where quality ratings below 6 are bad [0], and ratings of 6 and above are good [1]

In [14]:
# YOUR CODE HERE
y = df['quality rating'].map(lambda x: 0 if x < 6 else 1)


❓ Check the class balance of the new binary target

In [15]:
# YOUR CODE HERE
y.value_counts()

0    50010
1    49990
Name: quality rating, dtype: int64

❓ Create your `X` by normalising the features. This will allow for fair comparison of different solvers.

In [16]:
# YOUR CODE HERE
X = df.drop(columns='quality rating')

In [18]:
# YOUR CODE HERE
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = X.columns

## 2. LogisticRegression solvers

❓ Logistic Regression models can be optimized using different **solvers**. Make a comparison of the available solvers':
- Fit time - which solver is **the fastest**?
- Precision - **how different** are their respective precision scores?

Available solvers for Logistic Regression are `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
For more information on these 5 solvers, check out [this Stack Overflow thread](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

In [26]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

solvers= ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
fit_time = []
Precision = []

for x in solvers:
    model_log_reg = LogisticRegression(solver=x, max_iter=1000)
    x = cross_validate(model_log_reg, X_scaled, y, cv=10, scoring='precision')
    fit_time.append(x['fit_time'].mean())
    Precision.append(x['test_score'].mean())

In [27]:
fit_time

[0.3382686376571655,
 0.3423058271408081,
 0.2305824041366577,
 0.6785547494888305,
 1.173775863647461]

In [28]:
# YOUR ANSWER
fastest_solver = "liblinear"

<details>
    <summary>ℹ️ Click here for our interpretation</summary>

All solvers should produce similar precision scores because our cost-function is "easy" enough to have a global minimum which is found by all 5 solvers. For very complex cost-functions such as in Deep Learning, different solvers may stopping at different values of the loss function.

**The wine dataset**
    
If you check feature importance with sklearn's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> on the current dataset, you'll see many features result in almost 0 importance. Liblinear solver successively moves only along *one* direction at a time, regularizing the others with L1 regularization (a.k.a, setting their beta to 0), which might provide a good fit for a dataset where many features are not that important in predicting the target.

❗️There is a cost to searching for the best solver. Sticking with the default (`lbfgs`) may save the most time overall, sklearn provides you this grid for an idea of which solver to choose to start off with: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers-chart.png" width=700>



</details> 

In [30]:
Precision

[0.8742370014407493,
 0.8742344676126175,
 0.8743399285518076,
 0.8742159423885985,
 0.8742370014407493]

###  🧪 Test your code

In [29]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-7.1.3, pluggy-1.0.0 -- /Users/bita/.pyenv/versions/lewagon_current/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bita/code/lewagon/data-solvers/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.07s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stochastic Gradient Descent

Logistic Regression models can also be optimized via Stochastic Gradient Descent.

❓ Evaluate a Logistic Regression model optimized via **Stochastic Gradient Descent**. How do its precision score and training time compare to the performance of the models trained in section 2?


<details>
<summary>💡 Hint</summary>

- If you are stuck, look at the [SGDClassifier doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)!

</details>



In [31]:
# YOUR CODE HERE
from sklearn.linear_model import SGDClassifier

solvers= ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
fit_time = []
Precision = []

model_sgd_classifier = SGDClassifier()

for x in solvers:
    model_log_reg = LogisticRegression(solver=x, max_iter=1000)
    x = cross_validate(model_sgd_classifier, X_scaled, y, cv=10, scoring='precision')
    fit_time.append(x['fit_time'].mean())
    Precision.append(x['test_score'].mean())

In [32]:
fit_time

[0.1762096643447876,
 0.19046370983123778,
 0.20328178405761718,
 0.17328157424926757,
 0.18087735176086425]

In [33]:
Precision

[0.8837513180049648,
 0.882299076728081,
 0.8794460586567867,
 0.8892483691503413,
 0.8868936551195945]

☝️ The SGD model should have one of the shortest times (maybe even shorter than `liblinear`), for similar performance. This is a direct effect of performing each epoch of the Gradient Descent on a single row as opposed to loading 100k rows into memory at a time.

## 4. Predictions

❓ Use the best model (balanced with short fit time and high precision) to predict the binary quality (0 or 1) of the following wine. Store your:
- `predicted_class`
- `predicted_proba_of_class` (i.e if your model predicted a class of 1 what is the probability it believes 1 to be the class should be between 0 and 1)

In [34]:
new_wine = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_new_wine.csv')
new_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,9.54,13.5,12.35,8.78,14.72,9.06,9.67,10.15,11.17,12.17


In [35]:
# YOUR CODE HERE
from sklearn.model_selection import cross_val_predict, cross_val_score

best_model = SGDClassifier(loss="log_loss").fit(X_scaled,y)

new_X = pd.DataFrame(scaler.transform(new_wine))
new_X.columns = new_wine.columns

best_model.predict(new_X)

array([0])

In [36]:
predicted_class = best_model.predict(new_X)[0]
predicted_class

0

In [37]:
predicted_proba_of_class = best_model.predict_proba(new_X)[0, predicted_class]
predicted_proba_of_class

0.9687045491298731

# 🏁  Check your code and push your notebook

In [38]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-7.1.3, pluggy-1.0.0 -- /Users/bita/.pyenv/versions/lewagon_current/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bita/code/lewagon/data-solvers/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.20s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

